# Парсим

In [1]:
import json
import pandas as pd
from itertools import chain
import requests
import time
from tqdm import tqdm
# from ratelimit import limits, sleep_and_retry
import datetime
import os
import logging
import random
logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.DEBUG)

date = str(datetime.date.today())

In [7]:
# vk-api token
TOKEN = ""

# from "https://vk.com/promokody_samokat" take "promokody_samokat"
event_id = "promokody_samokat"

# better fix date
date = '2023-11-17'

In [8]:
data_folder = '../../data/vk-graph'
sure_members_path = f'{data_folder}/{event_id}/{date}/sure_members.json'
unsure_members_path = f'{data_folder}/{event_id}/{date}/unsure_members.json'
group_info_path= f'{data_folder}/{event_id}/{date}/group_info.json'
friends_path = f'{data_folder}/{event_id}/{date}/friends.jsonl'
followers_path = f'{data_folder}/{event_id}/{date}/followes.jsonl'
subscriptions_people_path = f'{data_folder}/{event_id}/{date}/sub_people.jsonl'
subscriptions_groups_path = f'{data_folder}/{event_id}/{date}/sub_groups.jsonl'
posts_path = f'{data_folder}/{event_id}/{date}/posts.jsonl'
likes_path = f'{data_folder}/{event_id}/{date}/likes.jsonl'
top_gropus_path = f'{data_folder}/{event_id}/{date}/top_groups.jsonl'
group_posts_path = f'{data_folder}/{event_id}/{date}/group_posts.jsonl'
group_likes_path = f'{data_folder}/{event_id}/{date}/group_likes.jsonl'

os.makedirs(f"{data_folder}/{event_id}/{date}", exist_ok=True)
ts_date = time.mktime(datetime.datetime.strptime(date, "%Y-%m-%d").timetuple())

post_time_gap = 365 * 3600 * 24

user_fields = ['id', 'first_name', 'last_name', 'deactivated', 'is_closed', 'about', 'activities', 'bdate', 'blacklisted', 'blacklisted_by_me', 'books', 'can_post', 'can_see_all_posts', 'can_see_audio', 'can_send_friend_request', 'can_write_private_message', 'career', 'city', 'common_count', 'connections', 'contacts', 'counters', 'country', 'domain', 'education', 'exports', 'followers_count', 'friend_status', 'games', 'has_mobile', 'has_photo', 'home_town', 'interests', 'is_no_index', 'last_seen', 'lists', 'maiden_name', 'military', 'movies', 'music', 'nickname', 'occupation', 'online', 'personal', 'photo_max_orig', 'quotes', 'relatives', 'relation', 'schools', 'screen_name', 'sex', 'site', 'status', 'timezone', 'trending', 'tv', 'universities', 'verified', 'wall_default']
group_fields = ['id', 'name', 'screen_name', 'is_closed', 'deactivated', 'type', 'photo_200', 'activity', 'addresses', 'age_limits', 'ban_info', 'can_create_topic', 'can_message', 'can_post', 'can_suggest', 'can_see_all_posts', 'can_upload_doc', 'can_upload_story', 'can_upload_video', 'city', 'contacts', 'counters', 'country', 'cover', 'description', 'fixed_post', 'has_photo', 'is_messages_blocked', 'links', 'main_album_id', 'main_section', 'market', 'members_count', 'place', 'public_date_label', 'site', 'start_date', 'finish_date', 'status', 'trending', 'verified', 'wall', 'wiki_page']
# user_fields = ['id', 'first_name', 'last_name', 'deactivated', 'is_closed', 'about', 'activities', 'bdate', 'blacklisted', 'blacklisted_by_me', 'books', 'can_post', 'can_see_all_posts', 'can_see_audio', 'can_send_friend_request', 'can_write_private_message', 'career', 'city', 'common_count', 'connections', 'contacts', 'counters', 'country', 'domain', 'education', 'exports', 'followers_count', 'friend_status', 'games', 'has_mobile', 'has_photo', 'home_town', 'interests', 'is_no_index',]
# group_fields = ['id', 'name', 'screen_name', 'is_closed', 'deactivated', 'type', 'photo_200', 'activity', 'addresses', 'age_limits', 'ban_info', 'can_create_topic', 'can_message', 'can_post', 'can_suggest', 'can_see_all_posts', 'can_upload_doc', 'can_upload_story', 'can_upload_video', 'city', 'contacts', 'counters', 'country', 'cover', 'description', 'fixed_post', 'has_photo', 'is_messages_blocked',]

In [9]:
# @sleep_and_retry
# @limits(calls=CALLS, period=PERIOD)
req_cnt = 0
def call_api(url: str, params: dict):
    global req_cnt
    req_cnt += 1
    def_params = {
        "access_token": TOKEN,
        "v": "5.131"
    }
    def_params.update(params)
    ans = requests.get(url, def_params).json()
    
    if 'error' in ans:
        if ans.get('error').get('error_code') in [10, 29]:
            delay = 1*60*60
            print(f"{ans},\n wait {delay//60}m")
            time.sleep(delay)
            return call_api(url, def_params)
    
    time.sleep(0.25 + 0.2 * random.random())

    return ans

In [10]:
%%time
url = "https://api.vk.com/method/groups.getById"
params = {
    "group_id": event_id,
    "fields": ','.join(group_fields),
}
req = call_api(url, params).get('response')[0]
is_event = req.get('type') == 'event'
with open(group_info_path, 'w') as f:
            json.dump(req, f)

offset = 0
c = 1000
user_list = []
url = "https://api.vk.com/method/groups.getMembers"
is_first = True
while True:
    params = {
        "group_id": event_id,
        "fields": ",".join(user_fields),
        "offset": offset,
        "count": c,
    }
    req = call_api(url, params)
    if 'error' in req:
        if req.get('error').get('error_code') in [18, 30]:
            raise Exception("error:" + str(req))
        else:
            raise Exception("Unknown_error:" + str(req))
    if is_first:
        print(f"User count: {req.get('response').get('count')}")
        is_first = False
    print(f"Itrerations: {offset//c + 1}/{req.get('response').get('count')//c + 1}")
    user_list += req.get('response').get('items')
    if (offset + c) >= req.get('response').get('count'):
        break
    else:
        offset += c
sure_members = user_list
if user_list:
    with open(sure_members_path, 'w') as f:
        json.dump(user_list, f)


if is_event:
    offset = 0
    c = 1000
    user_list = []
    url = "https://api.vk.com/method/groups.getMembers"
    is_first = True
    while True:
        params = {
            "group_id": event_id,
            "fields": ",".join(user_fields),
            "offset": offset,
            "count": c,
            "filter": "unsure",
        }
        req = call_api(url, params)
        if 'error' in req:
            if req.get('error').get('error_code') in [18, 30]:
                raise Exception("error:" + str(req))
            else:
                raise Exception("Unknown_error:" + str(req))
        if is_first:
            print(f"User count: {req.get('response').get('count')}")
            is_first = False
        print(f"Itrerations: {offset//c + 1}/{req.get('response').get('count')//c + 1}")
        user_list += req.get('response').get('items')
        if (offset + c) >= req.get('response').get('count'):
            break
        else:
            offset += c
    unsure_members = user_list
    if user_list:
        with open(unsure_members_path, 'w') as f:
            json.dump(user_list, f)

User count: 12562
Itrerations: 1/13
Itrerations: 2/13
Itrerations: 3/13
Itrerations: 4/13
Itrerations: 5/13
Itrerations: 6/13
Itrerations: 7/13
Itrerations: 8/13
Itrerations: 9/13
Itrerations: 10/13
Itrerations: 11/13
Itrerations: 12/13
Itrerations: 13/13
CPU times: total: 11 s
Wall time: 2min 21s


In [11]:
with open(sure_members_path) as f:
    sure_members = json.load(f)
with open(group_info_path) as f:
    group_info = json.load(f)

if group_info.get('type') == 'event':
    with open(unsure_members_path) as f:
        unsure_members = json.load(f)
    members = sure_members + unsure_members
else:
    members = sure_members

In [ ]:
%%time
start_with = None
is_achived = False
for member in tqdm(members[:]):
    if member.get("is_closed"):
        continue
    if not is_achived and start_with:
        if member.get('id') == start_with:
            is_achived = True
        continue
    
    # get friends
    offset = 0
    c = 5000
    user_list = []
    url = "https://api.vk.com/method/friends.get"
    while True:
        params = {
            "user_id": member.get('id'),
            "offset": offset,
            "count": c,
            # "fields": ",".join(user_fields),
        }
        req = call_api(url, params)
        if 'error' in req:
            if req.get('error').get('error_code') in [18, 30, 15]:
                break
            else:
                print(req)
                exit()
        if user_list:
            friend_set = set(user_list)
            new_set = set(req.get('response').get('items'))
            if len(new_set) == 0:
                break
            if len(friend_set.intersection(new_set)) > 0:
                break
            else:
                user_list += req.get('response').get('items')
                if len(req.get('response').get('items')) < c:
                    break
                offset += c
        else:
            user_list += req.get('response').get('items')
            if req.get('response').get('count') < c:
                break
            else:
                offset += c
    if user_list:
        with open(friends_path, 'a') as f:
            json.dump({member.get('id'): user_list}, f)
            f.write('\n')
    
    # get subscriptions profile, page, group
    # if subs 1000+ save only first 1000
    user_list = []
    group_list = []
    url = "https://api.vk.com/method/users.getSubscriptions"
    params = {
        "user_id": member.get('id'),
    }
    req = call_api(url, params)
    if 'error' in req:
        if req.get('error').get('error_code') in [18, 30, 15]:
            continue
        else:
            print(req)
            exit()
    user_list += req.get('response').get('users').get('items')
    group_list += req.get('response').get('groups').get('items')
    logging.debug(f"{req.get('response').keys()}, {[req.get('response').get(i).get('count') for i in req.get('response').keys()]}")

    if user_list:
        with open(subscriptions_people_path, 'a') as f:
            json.dump({member.get('id'): user_list}, f)
            f.write('\n')
    if group_list:
        with open(subscriptions_groups_path, 'a') as f:
            json.dump({member.get('id'): group_list}, f)
            f.write('\n')
    
    # get first 100 posts
    post_list = []
    url = "https://api.vk.com/method/wall.get"
    params = {
        "owner_id": member.get('id'),
    }
    req = call_api(url, params)
    if 'error' in req:
        if req.get('error').get('error_code') in [18, 30, 15]:
            continue
        else:
            print('error: ', req)
            exit()
    post_list += req.get('response').get('items')

    if post_list:
        with open(posts_path, 'a') as f:
            json.dump({member.get('id'): post_list}, f)
            f.write('\n')
    else:
        continue
    
    # get likes
    for post in post_list:
        if post.get("date") + post_time_gap < ts_date:
            continue
        offset = 0
        c = 1000
        like_list = []
        while True:
            url = "https://api.vk.com/method/likes.getList"
            params = {
                "type": "post",
                "owner_id": post.get("from_id"),
                "offset": offset,
                "count": c,
                "item_id": post.get("id"),
            }
            req = call_api(url, params)
            if 'error' in req:
                if req.get('error').get('error_code') in [18, 30, 15]:
                    break
                else:
                    print('error: ', req)
                    exit()
            like_list += req.get('response').get('items')
            if (offset + c) >= req.get('response').get('count'):
                break
            else:
                offset += c

        if like_list:
            with open(likes_path, 'a') as f:
                json.dump({"user_id": post.get("from_id"),
                           "post_id": post.get("id"),
                           "likes": like_list}, f)
                f.write('\n')
print('req_cnt:', req_cnt)

 15%|███████████▌                                                               | 1942/12562 [46:04<3:34:40,  1.21s/it]

In [6]:
# load top groups info
n = 500 # don't change!!!(depends on vk api)
d_border = 15 # how many minimal subscribers per group to parse
with open(subscriptions_groups_path) as f:
    sub_groups = [json.loads(line).popitem()[1] for line in f]
subs = pd.Series(chain.from_iterable(sub_groups))
batches = subs.value_counts()[subs.value_counts() > d_border].index.map(str).to_list()
batches = [batches[i:i + n] for i in range(0, len(batches), n)]
for i in tqdm(batches):
    group_list = []
    url = "https://api.vk.com/method/groups.getById"
    params = {
        "group_ids": ','.join(i),
        "fields": ','.join(group_fields),
    }
    req = call_api(url, params)
    if 'error' in req:
        if req.get('error').get('error_code') in [18, 30, 15]:
            continue
        else:
            print('error: ', req)
            exit()
    # print(req)
    group_list += req.get('response')

    if group_list:
        with open(top_gropus_path, 'a') as f:
            for group in group_list:
                json.dump(group, f)
                f.write('\n')
    else:
        continue
print('req_cnt:', req_cnt)

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [05:01<00:00, 11.60s/it]

req_cnt: 26


In [12]:
%%time
# get last 100 psots and likes from main group

post_list = []
url = "https://api.vk.com/method/wall.get"
params = {
    "domain": event_id,
    # "extended": True,
    # "offset": offset,
    "count": 100,
    # "fields": ",".join(user_fields + group_fields),
}
req = call_api(url, params)
if 'error' in req:
    if req.get('error').get('error_code') in [18, 30, 15]:
        print('error: ', req.get('error').get('error_code'))
        exit()
    else:
        print('error: ', req.get('error').get('error_code'))
post_list += req.get('response').get('items')
# logging.debug(f"{req.get('response').keys()}, {[req.get('response').get(i).get('count') for i in req.get('response').keys()]}")

if post_list:
    with open(group_posts_path, 'a') as f:
        json.dump({event_id: post_list}, f)
        f.write('\n')
else:
    exit()

# get group likes
for post in tqdm(post_list):
    if post.get("date") + post_time_gap < ts_date:
        continue
    offset = 0
    c = 1000
    like_list = []
    while True:
        url = "https://api.vk.com/method/likes.getList"
        params = {
            "type": "post",
            "owner_id": post.get("from_id"),
            "offset": offset,
            "count": c,
            "item_id": post.get("id"),
            # "fields": ",".join(user_fields),
        }
        req = call_api(url, params)
        if 'error' in req:
            if req.get('error').get('error_code') in [18, 30, 15]:
                break
            else:
                print('error: ', req)
        like_list += req.get('response').get('items')
        if (offset + c) >= req.get('response').get('count'):
            break
        else:
            offset += c

    if like_list:
        with open(group_likes_path, 'a') as f:
            json.dump({"user_id": post.get("from_id"),
                       "post_id": post.get("id"),
                       "likes": like_list}, f)
            f.write('\n')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:20<00:00,  1.03it/s]

CPU times: total: 10.8 s
Wall time: 21.4 s


In [13]:
req_cnt

4780